# Analysis of ECCNP Resolution and Impact on NER and EL Performance

In [1]:
from xmen import load_config, load_kb
from xmen.data import from_spacy

conf = load_config('xmen_ggponc3.yaml')

**Note:** For this notebook, we only use a small subset of GGPONC 2.0 for demonstration purposes
Change the version if you want to use any other GGPONC release

In [2]:
from ggponc import *
import os

sent_df = read_sentences(os.path.join('data', conf.data_version), zipped=False)

  0%|          | 0/14 [00:00<?, ?it/s]

In [3]:
resolver = ECCNPResolver(**conf.eccnp)

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
sent_df = resolver.set_df(sent_df)

  0%|          | 0/374 [00:00<?, ?it/s]

In [5]:
ner = NERTagger()

In [6]:
ner_df_before = ner.set_df(sent_df, sent_col='sentence')
ds_ner_before = from_spacy(ner_df_before.spacy_ner, span_key='entities', doc_id_key='file_name')

  0%|          | 0/374 [00:00<?, ?it/s]

In [7]:
ner_df_after = ner.set_df(sent_df)
ds_ner_after = from_spacy(ner_df_after.spacy_ner, span_key='entities', doc_id_key='file_name')

  0%|          | 0/374 [00:00<?, ?it/s]

In [8]:
from ggponc import EntityLinker
linker = EntityLinker(**conf.linker.ranking, candidate_generation_kwargs=conf.linker.candidate_generation)

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_down

[06/19/24 13:43:53] INFO     Loading hierarchical faiss index                                ]8;id=178041;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/sap_bert_linker.py\sap_bert_linker.py]8;;\:]8;id=869901;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/sap_bert_linker.py#153\153]8;;\

                    INFO     Loading index from                                                 ]8;id=194356;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=496281;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py#64\64]8;;\
                             /home/Florian.Borchert/.cache/xmen/ggponc3/index/sapbert/embed_fai                    
                             ss_hier.pickle                                                                        

[06/19/24 13:43:56] INFO     Loaded index of type <class 'faiss.swigfaiss.IndexHNSWFlat'> and   ]8;id=828593;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=131882;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py#66\66]8;;\
                             size 2465719                                                                          

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
linking_results = {}
linking_results['before_eccnp'] = linker.transform(ds_ner_before, silent=True)
linking_results['before_eccnp_no_semtype'] = linker.transform(ds_ner_before, ['abbrv', 'candidates', 'reranking'], silent=True)
linking_results['after_eccnp'] = linker.transform(ds_ner_after, silent=True)
linking_results['after_eccnp_no_semtype'] = linker.transform(ds_ner_after, ['abbrv', 'candidates', 'reranking'], silent=True)

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

...

  0%|          | 0/1177 [00:00<?, ?it/s]

  0%|          | 0/1177 [00:00<?, ?it/s]

Batches:   0%|          | 0/1177 [00:00<?, ?it/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

In [10]:
from nb_util import show_diff_single
from IPython.display import display
from nen_util import get_annotation_dataframe
from xmen.data import Sampler

def get_ent_texts(d):
    return [e['text'] for e in d['entities']]

# Impact of Semantic Type Filter

In [11]:
res_sem = linking_results['before_eccnp']['reranked']
res_no_sem = linking_results['before_eccnp_no_semtype']['reranked']

assert len(res_sem) == len(res_no_sem)

diff_sem = 0

for d_b, d_a in zip(res_no_sem, res_sem):
    t_a = d_a['passages'][0]['text'][0]
    df1 = get_annotation_dataframe([d_b], linker.kb, 1, 0.0)
    df2 = get_annotation_dataframe([d_a], linker.kb, 1, 0.0)
    if len(df1) == 0:
        continue
    mask = df1.canonical != df2.canonical
    if mask.any():
        print(t_a + '\n')
        #show_diff_single(t_b, t_a)
        print('No Filter:')
        display(df1[mask])
        print('Filter:')
        display(df2[mask])
        diff_sem += 1
        print('####')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
diff_sem / len(res_no_sem)

0.13368983957219252

# Impact of ECCNPs

In [13]:
res_before = linking_results['before_eccnp']['reranked']
res_after = linking_results['after_eccnp']['reranked']

assert len(res_sem) == len(res_no_sem)

diff_text = 0
diff_ents = 0
for d_b, d_a in zip(res_before, res_after):
    if d_b['passages'] != d_a['passages']: 
        diff_text += 1
        if get_ent_texts(d_b) != get_ent_texts(d_a):
            diff_ents += 1
            t_a = d_a['passages'][0]['text'][0]
            t_b = d_b['passages'][0]['text'][0]
            print('Difference', diff_ents)
            show_diff_single(t_b, t_a)
            display(get_annotation_dataframe([d_b], linker.kb, 1, 0.0))
            display(get_annotation_dataframe([d_a], linker.kb, 1, 0.0))

Difference 1


Am ehesten werden bei Patienten mit Malignomen Mangelzustände von Vitamin D beobachtet [REF] und von einigen Autoren mit der Erkrankungsinzidenz und -prognose in Verbindung gebracht [REF] [REF] [REF] [REF].

Am ehesten werden bei Patienten mit Malignomen Mangelzustände von Vitamin D beobachtet [REF] und von einigen Autoren mit der Erkrankungsinzidenz und Erkrankungsprognose in Verbindung gebracht [REF] [REF] [REF] [REF].

  0%|          | 0/1 [00:00<?, ?it/s]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
0,01_magenkarzinom_0192_2,Malignomen,Diagnosis_or_Pathology,36,46,C0006826,[T191],Krebserkrankung,"[ngram, sapbert]",0.078375
1,01_magenkarzinom_0192_2,Mangelzustände von Vitamin D,Diagnosis_or_Pathology,47,75,C0042870,[T047],Vitamin-D-Mangel,"[ngram, sapbert]",0.071928
2,01_magenkarzinom_0192_2,Vitamin D,Nutrient_or_Body_Substance,66,75,C0042866,"[T121, T125, T127]",Vitamin D,"[ngram, sapbert]",0.077961
3,01_magenkarzinom_0192_2,Erkrankungsinzidenz,Diagnosis_or_Pathology,125,144,C0012655,[T201],Krankheitsanfälligkeit,[sapbert],0.065133
4,01_magenkarzinom_0192_2,-prognose,Other_Finding,149,158,C0033325,[T058],Prognose,"[ngram, sapbert]",0.092011


...

Im Sinne eines Recall-Phänomens kann eine Pneumonitis auch lange nach Bestrahlung nach Chemotherapie oder Immunotherapie z.B. mit Taxanen, Gemcitabin oder Tyrosinkinase- Inhibitoren (Erlotinib, Gefitinib, Sunitinib und Folgesubstanzen) auftreten [REF] [REF] [REF] [REF] [REF] [REF] [REF] [REF] [REF] [REF].

  0%|          | 0/1 [00:00<?, ?it/s]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
0,27_supportive-therapie_0765_11,Recall-Phänomens,Diagnosis_or_Pathology,15,31,C1262013,[T046],Recall phenomenon,"[ngram, sapbert]",0.117091
1,27_supportive-therapie_0765_11,Pneumonitis,Diagnosis_or_Pathology,42,53,C3714636,[T047],Pneumonitis,"[ngram, sapbert]",0.129935
2,27_supportive-therapie_0765_11,Bestrahlung,Therapeutic,70,81,C1522449,[T061],Strahlentherapie,[sapbert],0.114749
3,27_supportive-therapie_0765_11,Chemo-,Therapeutic,87,93,C3665472,[T061],Chemotherapie,"[ngram, sapbert]",0.106600
4,27_supportive-therapie_0765_11,Immunotherapie,Therapeutic,99,113,C0021083,[T061],Immuntherapie,"[ngram, sapbert]",0.126997
5,27_supportive-therapie_0765_11,Taxanen,Clinical_Drug,123,130,C0796419,[T109],Taxane,"[ngram, sapbert]",0.079232
6,27_supportive-therapie_0765_11,Gemcitabin,Clinical_Drug,132,142,C0045093,"[T114, T121]",2' Deoxy 2' Difluorocytidine,"[ngram, sapbert]",0.132524
7,27_supportive-therapie_0765_11,Tyrosinkinase-,Clinical_Drug,148,162,C0033681,"[T116, T126]",Protein-Tyrosin-Kinase,"[ngram, sapbert]",0.141131
8,27_supportive-therapie_0765_11,Tyrosinkinase- Inhibitoren,Clinical_Drug,148,174,C4521478,[T080],Tyrosine Protein Kinase Inhibitors,"[ngram, sapbert]",0.139909
9,27_supportive-therapie_0765_11,Erlotinib,Clinical_Drug,176,185,C1135135,"[T109, T121]",Erlotinib,"[ngram, sapbert]",0.139762


  0%|          | 0/1 [00:00<?, ?it/s]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
0,27_supportive-therapie_0765_11,Recall-Phänomens,Diagnosis_or_Pathology,15,31,C1262013,[T046],Recall phenomenon,"[ngram, sapbert]",0.121586
1,27_supportive-therapie_0765_11,Pneumonitis,Diagnosis_or_Pathology,42,53,C3714636,[T047],Pneumonitis,"[ngram, sapbert]",0.129758
2,27_supportive-therapie_0765_11,Bestrahlung,Therapeutic,70,81,C1522449,[T061],Strahlentherapie,[sapbert],0.116514
3,27_supportive-therapie_0765_11,Chemotherapie,Therapeutic,87,100,C3665472,[T061],Chemotherapie,"[ngram, sapbert]",0.108146
4,27_supportive-therapie_0765_11,Immunotherapie,Therapeutic,106,120,C0021083,[T061],Immuntherapie,"[ngram, sapbert]",0.125116
5,27_supportive-therapie_0765_11,Taxanen,Clinical_Drug,130,137,C0796419,[T109],Taxane,"[ngram, sapbert]",0.077322
6,27_supportive-therapie_0765_11,Gemcitabin,Clinical_Drug,139,149,C0045093,"[T114, T121]",2' Deoxy 2' Difluorocytidine,"[ngram, sapbert]",0.135344
7,27_supportive-therapie_0765_11,Tyrosinkinase-,Clinical_Drug,155,169,C0033681,"[T116, T126]",Protein-Tyrosin-Kinase,"[ngram, sapbert]",0.134931
8,27_supportive-therapie_0765_11,Tyrosinkinase- Inhibitoren,Clinical_Drug,155,181,C4521478,[T080],Tyrosine Protein Kinase Inhibitors,"[ngram, sapbert]",0.137750
9,27_supportive-therapie_0765_11,Erlotinib,Clinical_Drug,183,192,C1135135,"[T109, T121]",Erlotinib,"[ngram, sapbert]",0.140089
